In [14]:
from operator import is_
import os
import sys
sys.path.append('../.')
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
import numpy as np

import clip

from Metrics import base_kmeans_model_evaluation, kmeans_with_init, cosine_kmeans_with_init
from networks import CustomCLIP, load_clip_to_cpu
from lr_scheduler import ConstantWarmupScheduler
from torch.utils.data import Dataset, DataLoader
import argparse

batch_size = 10
class custom_dataset(Dataset): 
    def __init__(self,  root_dir='../data/imagenet10', transform=None): 
        self.transform = transform
        data = np.load(os.path.join(root_dir,'data.npy'))
        self.targets = np.load(os.path.join(root_dir,'label.npy'))
        self.classes = list(set(self.targets))
        # b,h,w,c = data.shape
        # data = data.reshape((b, c, h, w))
        # self.data = data.transpose((0, 2, 3, 1))
        self.data = data
                
    def __getitem__(self, index):
        img, label = self.data[index], self.targets[index]
        img = Image.fromarray(img)
        img = self.transform(img) 
        return img,label
    def __len__(self):
        return len(self.data)
    
def dataset_select(dataset_name):
    if dataset_name == 'imagenet_dog':
        is_test_dataset = True
        fixed_dataset = custom_dataset(root_dir='../data/imagenet_dog', transform=preprocess)
        fixed_dataloader = DataLoader(
            dataset=fixed_dataset, 
            batch_size=batch_size,
            shuffle=False)
        dataset = custom_dataset(root_dir='../data/imagenet_dog', transform=preprocess)
        classes = ['cavachon','cavalier king charles spaniel','Basset Hound','Norwegian Elkhound',\
                  'Giant Schnauzer','golden retriever','brittany spaniel','clumber spaniel',\
                  'cocker spaniel','Belgian Shepherd Groenendael','Lancashire Heeler','rough collie','doberman pinscher','pug','chow chow']
    elif dataset_name == 'stanford40':
        is_test_dataset = True
        fixed_dataset = custom_dataset(root_dir='/database/daehyeon/Stanford40/JPEGImages', transform=preprocess)
        fixed_dataloader = DataLoader(
            dataset=fixed_dataset, 
            batch_size=batch_size,
            shuffle=False)
        dataset = custom_dataset(root_dir='/database/daehyeon/Stanford40/JPEGImages', transform=preprocess)
        classes = ['rowing a boat', 'blowing bubbles', 'phoning', 'playing guitar', 'fishing', 'pouring liquid', 'waving hands', 'writing on a book', 'holding an umbrella', 'riding a horse', 'washing dishes', 'fixing a bike', 'running', 'fixing a car', 'brushing teeth', 'climbing', 'cooking', 'jumping', 'looking through a microscope', 'feeding a horse', 'using a computer', 'reading', 'riding a bike', 'walking the dog', 'gardening', 'texting message', 'throwing frisby', 'cleaning the floor', 'drinking', 'pushing a cart', 'watching TV', 'cutting trees', 'looking through a telescope', 'cutting vegetables', 'shooting an arrow', 'smoking', 'applauding', 'writing on a board', 'taking photos', 'playing violin']

    elif dataset_name == 'mit':
        is_test_dataset = True
        fixed_dataset = custom_dataset(root_dir='/database/daehyeon/mit', transform=preprocess)
        fixed_dataloader = DataLoader(
            dataset=fixed_dataset, 
            batch_size=batch_size,
            shuffle=False)
        dataset = custom_dataset(root_dir='/database/daehyeon/mit', transform=preprocess)
        classes = ['poolinside','bedroom','inside_subway','operating_room','warehouse','clothingstore','toystore','waitingroom','lobby','casino','subway','closet','livingroom','laundromat','studiomusic','fastfood_restaurant','buffet','grocerystore','bakery','nursery','florist','airport_inside','artstudio','laboratorywet','videostore','kitchen','elevator','mall','deli','dentaloffice','greenhouse','bar','prisoncell','children_room','museum','gym','office','jewelleryshop','locker_room','hairsalon','restaurant','trainstation','pantry','dining_room','meeting_room','bookstore','tv_studio','bathroom','auditorium','bowling','shoeshop','restaurant_kitchen','hospitalroom','concert_hall','stairscase','corridor','garage','inside_bus','winecellar','computerroom','cloister','movietheater','kindergarden','library','classroom','gameroom','church_inside']
        trainset = torchvision.datasets.STL10(
        root='../data', split='train', download=True,
        transform=preprocess)
    
    elif dataset_name == 'stl10':
        testset = torchvision.datasets.STL10(
            root='../data', split='test', download=True,
            transform=preprocess)
        trainset = torchvision.datasets.STL10(
            root='../data', split='train', download=True,
            transform=preprocess)
        dataset = torch.utils.data.ConcatDataset([trainset, testset])
        classes = testset.classes
    elif dataset_name == 'cifar10':
        trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                       download=True, transform=preprocess)        
        testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=preprocess)
        dataset = torch.utils.data.ConcatDataset([trainset, testset])
        classes = testset.classes
    return dataset,classes

In [22]:
batch_size =100
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=preprocess)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
#                                          shuffle=False)

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                       download=True, transform=preprocess)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
#                                          shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# batch_size = 50
# trainset = torchvision.datasets.STL10(
#     root='../data', split='train', download=True,
#     transform=preprocess)
# testset = torchvision.datasets.STL10(
#     root='../data', split='test', download=True,
#     transform=preprocess)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
#                                           shuffle=False)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
#                                           shuffle=False)
# concatset = torch.utils.data.ConcatDataset([trainset, testset])
# concatloader = torch.utils.data.DataLoader(concatset, batch_size=batch_size,
#                                          shuffle=False)



In [15]:
from PIL import Image
dataset_name_list = ['cifar10']
for dataset_name in dataset_name_list:
    pred_list = []
    clip_backbone = "ViT-B/16"
    device = torch.device('cuda') if torch.cuda.is_available() else "cpu"
    backbone_name = clip_backbone
    clip_model, preprocess = load_clip_to_cpu(backbone_name)
    testset, classes = dataset_select(dataset_name)
    model = CustomCLIP(clip_model, len(classes))
    model.to(device)
    model.eval()
    classes_list = ['a photo of a '+ c for c in classes]
    dataloader = DataLoader(
        dataset=testset, 
        batch_size=batch_size,
        shuffle=False)
    classes_list = clip.tokenize(classes_list).to(device)
    centroid_candidate_text = clip_model.to(device).encode_text(classes_list)
    centroid_candidate_text = centroid_candidate_text / centroid_candidate_text.norm(dim=-1, keepdim=True)
    with torch.no_grad():
        model.eval()
        for i, data in enumerate(dataloader, 0):
            inputs, targets = data
            inputs, targets = inputs.to(device), targets.to(device)
            image_features = model.image_encoder(inputs.type(model.dtype))
            image_features = image_features / \
            image_features.norm(dim=-1, keepdim=True)

            logit = image_features @ centroid_candidate_text.t()
            pred = torch.argmax(logit, dim=1)
            pred_list.append(pred)
            if i%50 == 0:
                print(i,'/',len(dataloader))

    pred_list = np.array(pred_list)
    
    for i,item in enumerate(pred_list):
        if i == 0:
            pred_tensor = item.cpu()
        else:
            pred_tensor = torch.concat((pred_tensor,item.cpu()),axis=0)  
    acc = torch.sum(pred_tensor==torch.tensor(testset.targets))/len(testset.targets)

    print(dataset_name,'accuracy: {}'.format(acc))

Files already downloaded and verified
Files already downloaded and verified
Initializing class-specific contexts
Initial context: "age of personX X X X X X X X X X X X X X X X"
Number of context words (tokens): 16
0 / 6000
50 / 6000
100 / 6000
150 / 6000
200 / 6000
250 / 6000
300 / 6000
350 / 6000
400 / 6000
450 / 6000
500 / 6000
550 / 6000
600 / 6000
650 / 6000
700 / 6000
750 / 6000
800 / 6000
850 / 6000
900 / 6000
950 / 6000
1000 / 6000
1050 / 6000
1100 / 6000
1150 / 6000
1200 / 6000
1250 / 6000
1300 / 6000
1350 / 6000
1400 / 6000
1450 / 6000
1500 / 6000
1550 / 6000
1600 / 6000
1650 / 6000
1700 / 6000
1750 / 6000
1800 / 6000
1850 / 6000
1900 / 6000
1950 / 6000
2000 / 6000
2050 / 6000
2100 / 6000
2150 / 6000
2200 / 6000
2250 / 6000
2300 / 6000
2350 / 6000
2400 / 6000
2450 / 6000
2500 / 6000
2550 / 6000
2600 / 6000
2650 / 6000
2700 / 6000
2750 / 6000
2800 / 6000
2850 / 6000
2900 / 6000
2950 / 6000
3000 / 6000
3050 / 6000
3100 / 6000
3150 / 6000
3200 / 6000
3250 / 6000
3300 / 6000
3350 

AttributeError: 

In [30]:
targets = trainset.targets + testset.targets
acc = torch.sum(pred_tensor==torch.tensor(targets))/len(targets)
print(acc)

classes_list = ['a photo of a '+ c for c in classes]
print(classes_list)



tensor(0.8957)
['a photo of a airplane', 'a photo of a automobile', 'a photo of a bird', 'a photo of a cat', 'a photo of a deer', 'a photo of a dog', 'a photo of a frog', 'a photo of a horse', 'a photo of a ship', 'a photo of a truck']


In [32]:

testset = torchvision.datasets.STL10(
    root='../data', split='test', download=True,
    transform=preprocess)
trainset = torchvision.datasets.STL10(
    root='../data', split='train', download=True,
    transform=preprocess)
dataset = torch.utils.data.ConcatDataset([trainset, testset])

testset.classes


Files already downloaded and verified
Files already downloaded and verified


['airplane',
 'bird',
 'car',
 'cat',
 'deer',
 'dog',
 'horse',
 'monkey',
 'ship',
 'truck']

In [46]:
labels = np.array(trainset.labels.tolist() + testset.labels.tolist())
acc = torch.sum(pred_tensor==torch.tensor(labels))/len(labels)
acc

tensor(0.9816)

In [ ]:

from PIL import Image
dataset_name_list = ['stl10']
pred_list = []
for dataset_name in dataset_name_list:
    clip_backbone = "ViT-B/16"
    device = torch.device('cuda') if torch.cuda.is_available() else "cpu"
    backbone_name = clip_backbone
    clip_model, preprocess = load_clip_to_cpu(backbone_name)
    testset, classes = dataset_select(dataset_name)
    model = CustomCLIP(clip_model, len(classes))
    model.to(device)
    model.eval()
    classes_list = ['a photo of a '+ c for c in classes]
    dataloader = DataLoader(
        dataset=testset, 
        batch_size=batch_size,
        shuffle=False)
    classes_list = clip.tokenize(classes_list).to(device)
    centroid_candidate_text = clip_model.to(device).encode_text(classes_list)
    centroid_candidate_text = centroid_candidate_text / centroid_candidate_text.norm(dim=-1, keepdim=True)
    with torch.no_grad():
        model.eval()
        for i, data in enumerate(dataloader, 0):
            inputs, targets = data
            inputs, targets = inputs.to(device), targets.to(device)
            image_features = model.image_encoder(inputs.type(model.dtype))
            image_features = image_features / \
            image_features.norm(dim=-1, keepdim=True)

            logit = image_features @ centroid_candidate_text.t()
            pred = torch.argmax(logit, dim=1)
            pred_list.append(pred)
            if i%50 == 0:
                print(i,'/',len(dataloader))

    pred_list = np.array(pred_list)
    acc = torch.mean(pred_list[0].cpu()==torch.tensor(testset.targets))
    print(dataset_name,'accuracy: {}'.format(acc))
    

In [59]:
lr = 1e-6
criterion = nn.MSELoss()

dataset_name_list = ['mit','stanford40','imagenet_dog']
for dataset_name in dataset_name_list:
    clip_backbone = "ViT-B/16"
    device = torch.device('cuda') if torch.cuda.is_available() else "cpu"
    backbone_name = clip_backbone
    clip_model, preprocess = load_clip_to_cpu(backbone_name)

    testset, classes = dataset_select(dataset_name)
    
    model = CustomCLIP(clip_model, len(classes))
    model.to(device)

    classes_list = ['a photo of a '+ c for c in classes]
    print(classes_list)
    with torch.no_grad():
        classes_list = clip.tokenize(classes_list).to(device)
        centroid_candidate_text = clip_model.to(device).encode_text(classes_list)
        centroid_candidate_text = centroid_candidate_text / centroid_candidate_text.norm(dim=-1, keepdim=True)

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=lr,
        weight_decay=5e-4,
        betas=(0.9, 0.999),
    )
    for i in range(1000):
        if i%50 == 0:
            print(i,'/',1000)
        prompts = model.prompt_learner().cuda()
        tokenized_prompts = model.tokenized_prompts
        for j in range(len(prompts)):
            text_feature = model.text_encoder(prompts[j], tokenized_prompts)
            text_feature = text_feature / \
                text_feature.norm(dim=-1, keepdim=True)
            if j == 0:
                text_features = text_feature.expand(1, -1, -1)
            else:
                text_feature = text_feature.expand(1, -1, -1)
                text_features = torch.cat((text_features, text_feature), dim=0)
        text_features = text_features.mean(dim=0)

        loss = criterion(text_features,centroid_candidate_text)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        knn = cosine_kmeans_with_init
        new_label, acc, nmi,ari = knn(
            model, testloader, num_classes, text_features)
    

Initializing class-specific contexts
Initial context: "age of personX X X X X X X X X X X X X X X X"
Number of context words (tokens): 16
['a photo of a poolinside', 'a photo of a bedroom', 'a photo of a inside_subway', 'a photo of a operating_room', 'a photo of a warehouse', 'a photo of a clothingstore', 'a photo of a toystore', 'a photo of a waitingroom', 'a photo of a lobby', 'a photo of a casino', 'a photo of a subway', 'a photo of a closet', 'a photo of a livingroom', 'a photo of a laundromat', 'a photo of a studiomusic', 'a photo of a fastfood_restaurant', 'a photo of a buffet', 'a photo of a grocerystore', 'a photo of a bakery', 'a photo of a nursery', 'a photo of a florist', 'a photo of a airport_inside', 'a photo of a artstudio', 'a photo of a laboratorywet', 'a photo of a videostore', 'a photo of a kitchen', 'a photo of a elevator', 'a photo of a mall', 'a photo of a deli', 'a photo of a dentaloffice', 'a photo of a greenhouse', 'a photo of a bar', 'a photo of a prisoncell', 

In [42]:
classes = ['poolinside','bedroom','inside_subway','operating_room','warehouse','clothingstore','toystore','waitingroom','lobby','casino','subway','closet','livingroom','laundromat','studiomusic','fastfood_restaurant','buffet','grocerystore','bakery','nursery','florist','airport_inside','artstudio','laboratorywet','videostore','kitchen','elevator','mall','deli','dentaloffice','greenhouse','bar','prisoncell','children_room','museum','gym','office','jewelleryshop','locker_room','hairsalon','restaurant','trainstation','pantry','dining_room','meeting_room','bookstore','tv_studio','bathroom','auditorium','bowling','shoeshop','restaurant_kitchen','hospitalroom','concert_hall','stairscase','corridor','garage','inside_bus','winecellar','computerroom','cloister','movietheater','kindergarden','library','classroom','gameroom','church_inside']


In [52]:
list_ = os.listdir('/database/daehyeon/temporary/')
dict_ = {}
for item in list_:
    a = item.split('.')[0]
    b = a.split('_')[:-1]
    c = ' '.join(b)
    dict_[c] = 1
classes = list(dict_.keys())
print(len(classes))
print(classes)

40
['rowing a boat', 'blowing bubbles', 'phoning', 'playing guitar', 'fishing', 'pouring liquid', 'waving hands', 'writing on a book', 'holding an umbrella', 'riding a horse', 'washing dishes', 'fixing a bike', 'running', 'fixing a car', 'brushing teeth', 'climbing', 'cooking', 'jumping', 'looking through a microscope', 'feeding a horse', 'using a computer', 'reading', 'riding a bike', 'walking the dog', 'gardening', 'texting message', 'throwing frisby', 'cleaning the floor', 'drinking', 'pushing a cart', 'watching TV', 'cutting trees', 'looking through a telescope', 'cutting vegetables', 'shooting an arrow', 'smoking', 'applauding', 'writing on a board', 'taking photos', 'playing violin']
